# 1. Project Overview

It is often not straight-forward to discern which securities have performed better than others when looking at historical data. Their market price changes constantly, with some events, such as economic recessions, pandemics or natural disasters heavily affecting the valuations of many companies.

However, these companies are not necessarily affected the same way, and their publicly traded price can, due to a multitude of factors, react very differently to the same events. For example, in 2020 there was the COVID-19 outbreak, eCommerce companies such as Amazon, as well as hardware manufacturers such as NVIDIA, saw a dramatic increase in the demand of their products due to the pandemic's lockdowns.

As small investors, we might be interested in analysing the past performance of a certain portfolio of companies, as well as be able forecast whether they will continue their present trend.

The goal of this project is to provide an easy to use interface to quickly compare the performance of multiple companies over a given period of time by means of visualizations and aggregated statistics. This will allow users to pick which companies did best, and which ones will in the future. We aim to answer questions such as:

1. Which company grew more in value?
2. Which company experimented the lowest volatility? Which experimented the highest?
3. Which company is expected to grow more in value?

However, it is important to consider that, according to [random walk theory](https://en.wikipedia.org/wiki/Random_walk_hypothesis), market prices behave randomly and not as a function of their time series. In other words, the patterns observed in the historical price a certain financial security are unlikely to help predict the future. In financial markets, we find that the future value of a security depends on its previous value plus some unexplained variance. Therefore, questions about what the future holds are not expected to be answered successfully.

> **DISCLAIMER:** This project is merely meant to be used for understanding the past and get a sense of the future. The insights gained and any recommendations made **are not financial advise**. The value of a company at any given time and its evolution dependes on many factors that aren't taken into account in this project. Real-world value investing requires an in-depth analysis of each company and sector, and it's still not guaranteed to yield better returns than simply investing in a market index. And above all, **Past performance is no guarantee of future results. Don't assume an investment will continue to do well in the future simply because it's done well in the past.**


## 1.1. Data Description

This project uses financial data extracted from [Yahoo Finance](https://finance.yahoo.com) through its API. For each set of *security tickers in an input portfolio*, the following data is downloaded:

- **Tickers information**: Contains multiple information fields of each ticker, such as company name, description and some financial metrics such as revenue.
- **Tickers daily data**: Daily price data price values (open, close, min, high) for each ticker for the user-provided data range.

## 1.2. Approach to solve the problem

We will follow along the CRISP-DM data science process to analyse the tickers in a pre-defined portfolio made up of several *big tech* companies. We already discussed the context of the problem and the questions we want to answer, as well as the data that we will be using. The remaining steps are as follows:

  1. **Data Exploration.** Downloading financial data using the Python package [yfinance](https://pypi.org/project/yfinance/) and exploring it.
  2. **Data Preparation.** Extracting insights from the data via statistics and visualizations.
  3. **Data Modeling.** Building a forecasting model to evaluate price predictions.

## 1.3. Expected solution

When analying the past price data, we expect that simple summary statistics such as relative price change over the selected time period or the price standard deviation will suffice to get a good sense of what companies performed best.

However, for future price prediction we expect to face difficulties in obtaining a model capable of accurately forecasting the future value of financial securities. This is especially true given that we will only be using historical price data.

## 1.4. Justification of metrics employed

The following main metrics will be employed to compare the market performance among securities:

- `rel_change`: A custom metric determining the evolution of the price with respect to its value at the beginning of the chosen date range.
- `std`: Standard deviation of the price, as a measure of volatility over the chosen date range.
- `max_fall`: Maximum price fall from a high peak. This can give a sense of the vulnerability of a security to fall from a high price point.
- `max_rise`: Maximum price rise from a low peak. This can give a sense of the ability of a security to rebound from a low price point.

Regarding forecasting, we will be using mean squared error to measure the goodness of our model fit.

## 2. Exploratory Data Analysis (EDA)

### Setup and imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import yfinance as yf
import pandas as pd
import logging
import sys
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

from copy import deepcopy
from pathlib import Path
from datetime import datetime
from dateutil.relativedelta import relativedelta
from lightgbm import LGBMRegressor
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.model_selection import grid_search_forecaster
from sklearn.metrics import mean_squared_error

In [3]:
src_path: str = "../src"
sys.path.append(src_path)
logging.basicConfig()
logging.getLogger().setLevel(logging.INFO)

In [4]:
from data.utils import download_yfinance_data, get_price_statistics
from data.plots import candlestick_yearly, violin_monthly, violin_weekday
from models.utils import split_time_data, preprocess_data, fit_forecaster
from models.plots import plot_data_split, plot_data_predictions

In [5]:
random_seed = 8080
portfolio_name = "big_tech"
data_path = Path("..").resolve().joinpath("data")

We define the portfolio we are going to use, as well as the data range we want to analyse.


In [6]:
portfolio_filepath = data_path.joinpath("portfolios").joinpath(f"{portfolio_name}.txt")
tickers = [line.split(" ")[0] for line in portfolio_filepath.read_text().split("\n")]
date_range = (
    datetime.now() - relativedelta(years=5),
    datetime.now(),
)

Some sections require using one single ticker, which is chosen below:

In [7]:
ticker_selected = tickers[0]

### 2.1. Tickers Information


We download the data using `yfinance`.

In [8]:
tickers_str = " ".join(tickers)

In [9]:
tickers = yf.Tickers(tickers_str)

tickers_info = []
for ticker_name, ticker_obg in tickers.tickers.items():
    try:
        tickers_info.append(pd.Series(ticker_obg.info).rename(ticker_name))
    except Exception as e:
        logging.warning(f"Problem retrieving information for {ticker_name}: {e}")
        continue

if len(tickers_info) != 0:
    tickers_info = pd.concat(tickers_info, axis=1)

In [10]:
tickers_info.dropna(how="all")

,GOOG,AMZN,AAPL,MSFT,NFLX
zip,94043,98109-5210,95014,98052-6399,95032
sector,Communication Services,Consumer Cyclical,Technology,Technology,Communication Services
fullTimeEmployees,190234,1541000,164000,221000,12800
longBusinessSummary,Alphabet Inc. offers various products and plat...,"Amazon.com, Inc. engages in the retail sale of...","Apple Inc. designs, manufactures, and markets ...","Microsoft Corporation develops, licenses, and ...","Netflix, Inc. provides entertainment services...."
city,Mountain View,Seattle,Cupertino,Redmond,Los Gatos
...,...,...,...,...,...
bidSize,1300,800,800,800,900
preMarketPrice,95.52,99.33,153.384,264.02,354.61
logo_url,https://logo.clearbit.com/abc.xyz,https://logo.clearbit.com/amazon.com,https://logo.clearbit.com/apple.com,https://logo.clearbit.com/microsoft.com,https://logo.clearbit.com/netflix.com
trailingPegRatio,1.1571,6.471,2.4749,2.2853,1.6656


In [11]:
print(tickers_info.dropna(how="all").index)

Index(['zip', 'sector', 'fullTimeEmployees', 'longBusinessSummary', 'city',
       'phone', 'state', 'country', 'companyOfficers', 'website', 'maxAge',
       'address1', 'industry', 'ebitdaMargins', 'profitMargins',
       'grossMargins', 'operatingCashflow', 'revenueGrowth',
       'operatingMargins', 'ebitda', 'targetLowPrice', 'recommendationKey',
       'grossProfits', 'freeCashflow', 'targetMedianPrice', 'earningsGrowth',
       'currentRatio', 'returnOnAssets', 'numberOfAnalystOpinions',
       'targetMeanPrice', 'debtToEquity', 'returnOnEquity', 'targetHighPrice',
       'totalCash', 'totalDebt', 'totalRevenue', 'totalCashPerShare',
       'financialCurrency', 'revenuePerShare', 'quickRatio',
       'recommendationMean', 'shortName', 'longName', 'isEsgPopulated',
       'gmtOffSetMilliseconds', 'messageBoardId', 'market',
       'enterpriseToRevenue', 'enterpriseToEbitda', 'forwardEps',
       'sharesOutstanding', 'bookValue', 'sharesShort',
       'sharesPercentSharesOut', 'la

Lots of current information is available for inspection for each of the security tickers in our portfolio. Among the many fields, we can find relevant financial indicators such as `ebitda`, `freeCashflow`, `revenuePerShare` and many, many others. Unfortunately, we don't have historical data for these fields, so we cannot link this information to our historical price data.

### 2.2. Tickers Historical Price data


In [12]:
start, end = date_range

In [13]:
try:
    tickers_data = yf.download(
        tickers_str, start=start, end=end, ignore_tz=True, keepna=True
    )
except Exception as e:
    tickers_data = pd.DataFrame()

[*********************100%***********************]  5 of 5 completed


In [14]:
tickers_data

Adj Close                                                 \
                  AAPL        AMZN       GOOG        MSFT        NFLX   
Date                                                                    
2018-02-20   40.957069   73.417503  55.123001   87.506783  278.549988   
2018-02-21   40.771172   74.146004  55.567001   86.345940  281.040009   
2018-02-22   41.111977   74.266998  55.331501   86.572464  278.140015   
2018-02-23   41.826962   75.000000  56.339500   88.771461  285.929993   
2018-02-26   42.653965   76.097504  57.187500   90.054985  294.160004   
...                ...         ...        ...         ...         ...   
2023-02-10  151.009995   97.610001  94.860001  262.442657  347.359985   
2023-02-13  153.850006   99.540001  95.000000  270.642120  358.570007   
2023-02-14  153.199997   99.699997  94.949997  271.490021  359.959991   
2023-02-15  155.330002  101.160004  97.099998  269.320007  361.420013   
2023-02-16  154.990005   99.120003  96.294998  263.649994  354.390015   

                 Close                                                 ...  \
                  AAPL        AMZN       GOOG        MSFT        NFLX  ...   
Date                                                                   ...   
2018-02-20   42.962502   73.417503  55.123001   92.720001  278.549988  ...   
2018-02-21   42.767502   74.146004  55.567001   91.489998  281.040009  ...   
2018-02-22   43.125000   74.266998  55.331501   91.730003  278.140015  ...   
2018-02-23   43.875000   75.000000  56.339500   94.059998  285.929993  ...   
2018-02-26   44.742500   76.097504  57.187500   95.419998  294.160004  ...   
...                ...         ...        ...         ...         ...  ...   
2023-02-10  151.009995   97.610001  94.860001  263.100006  347.359985  ...   
2023-02-13  153.850006   99.540001  95.000000  271.320007  358.570007  ...   
2023-02-14  153.199997   99.699997  94.949997  272.170013  359.959991  ...   
2023-02-15  155.330002  101.160004  97.099998  269.320007  361.420013  ...   
2023-02-16  154.990005   99.120003  96.294998  263.649994  354.390015  ...   

                  Open                                                \
                  AAPL       AMZN       GOOG        MSFT        NFLX   
Date                                                                   
2018-02-20   43.012501  72.324501  54.528500   91.480003  277.739990   
2018-02-21   43.207500  74.250000  55.323502   92.980003  282.070007   
2018-02-22   42.950001  74.767998  55.809502   92.050003  283.880005   
2018-02-23   43.417500  74.766998  55.632000   93.599998  281.000000   
2018-02-26   44.087502  75.459999  56.389999   94.400002  288.750000   
...                ...        ...        ...         ...         ...   
2023-02-10  149.460007  97.559998  95.739998  261.529999  359.160004   
2023-02-13  150.949997  97.849998  95.010002  267.640015  349.500000   
2023-02-14  152.119995  98.410004  94.660004  272.670013  357.549988   
2023-02-15  153.110001  99.089996  94.739998  268.320007  356.630005   
2023-02-16  153.509995  99.209999  95.540001  264.019989  355.000000   

               Volume                                           
                 AAPL       AMZN      GOOG      MSFT      NFLX  
Date                                                            
2018-02-20  135722000  129984000  28462000  30911700   7769000  
2018-02-21  149886400  126088000  30258000  26922500   9371100  
2018-02-22  123967600   97162000  26344000  24392800   8891500  
2018-02-23  135249600   88362000  25220000  26329200   7301800  
2018-02-26  152648800   99100000  31182000  30199800  10268600  
...               ...        ...       ...       ...       ...  
2023-02-10   57409100   52698600  49269900  25760900   7287300  
2023-02-13   62199000   52841500  43116600  44630900   7134400  
2023-02-14   61707600   56202900  42513100  37047900   4624800  
2023-02-15   65669300   48053900  37029900  28962200   3969900  
2023-02-16   45781750   38279153  26428962 

Different information is available for each date and ticker: `Adj Close`, `Close`, `High`, `Low`, `Open` and `Volume`. We will only be using `Adj Close` for performance analysis as well as forecasting. `NaN` value indicate periods for which no data is available.


## 3. Data Preprocessing

Here are the steps we need to follow for pre-processing the data:

1. Ensure that the index of our historical is of type `datetime` object and that it follows a daily frequency.
2. Leading and trailing `NaN` values are removed, whereas `NaN` values found in the middle of periods, such as weekends, will be forward filled. In the case of stock securities of publicly traded companies, we don't have values for Saturdays and Sundays, since the stock markets are closed. **This step is only done right before data modeling, and is done per ticker**.

In [15]:
tickers_data.index = pd.to_datetime(tickers_data.index, format="YYYY-MM-DD")
tickers_data = tickers_data.asfreq("D").sort_index()

### 3.1. Visualize price evolution

Naturally, because each security is valued at different scales, it is difficult to compare between companies.

In [16]:
fig = px.line(tickers_data["Adj Close"], title="Price evolution")
fig.update_layout(
    yaxis_title="Adj. Close price",
    xaxis_title="Date",
    legend_title="Security ticker",
)
fig.update_layout(template="seaborn").show()

### 3.2. Visualize price evolution (in relative terms)

When we plot the price evolution relative to its starting point, it is easier to see which security has done better over the chosen period.

In [17]:
def rel_change(series: pd.Series):
    """Relative price increases"""
    initial_value = series.dropna().iloc[0]
    return ((series - initial_value) / initial_value) * 100


fig = px.line(
    tickers_data["Adj Close"].apply(rel_change, axis=0),
    title="Price evolution (in relative terms from the start date)",
)
fig.update_layout(
    yaxis_title="Adj. Close price chage w.r.t. the start date",
    xaxis_title="Date",
    legend_title="Security ticker",
)
fig.update_layout(template="seaborn").show()

### 3.3. Historical Price Statistics

In [18]:
def abs_change(series: pd.Series) -> float:
    """Compute absolute change from first to last element in the series"""
    series = series.dropna()
    return series.iloc[-1] - series.iloc[0]


def rel_change(series: pd.Series) -> float:
    """Compute relative change from first to last element in the series"""
    series = series.dropna()
    return (abs_change(series) / series.iloc[0]) * 100


def max_fall(series: pd.Series) -> float:
    """Compute max fall from a previous all-time high"""
    series = series.dropna()
    return np.min(
        [
            (np.min(series.iloc[i:] - np.max(series.iloc[:i])))
            / np.max(series.iloc[:i])
            for i in range(1, len(series))
        ]
    )


def max_rise(series: pd.Series) -> float:
    """Compute max rise from a previous all-time low"""
    series = series.dropna()
    return np.max(
        [
            (np.max(series.iloc[i:] - np.min(series.iloc[:i])))
            / np.min(series.iloc[:i])
            for i in range(1, len(series))
        ]
    )

In [19]:
price_stats = (
    pd.concat(
        (
            tickers_data["Adj Close"].describe(),
            tickers_data["Adj Close"].agg([abs_change, rel_change, max_fall, max_rise]),
        )
    )
    .round(2)
    .transpose()
    .sort_values(by="rel_change", ascending=False)
)
price_stats

,count,mean,std,min,25%,50%,75%,max,abs_change,rel_change,max_fall,max_rise
AAPL,1258.0,99.29,46.24,34.26,50.46,109.11,143.08,180.68,114.03,278.42,-0.39,4.27
MSFT,1258.0,193.87,73.91,82.28,122.36,203.07,254.47,339.08,176.14,201.29,-0.37,3.12
GOOG,1258.0,87.03,30.90,48.81,59.22,75.81,113.34,150.71,41.17,74.69,-0.45,2.09
AMZN,1258.0,120.61,34.76,67.20,89.63,108.22,158.09,186.57,25.70,35.01,-0.56,1.78
NFLX,1258.0,390.43,117.27,166.37,307.31,362.87,494.56,691.69,75.84,27.23,-0.76,1.96


According to these statistics, we can see that **GOOG** experimented the least price volatily in the observed period (`std=30.91`), while **NFLX** experimented the largest (`std=117.31`).

On the other hand, **APPL** increased its value the most since the beginning of the observed period (`rel_change=277.76`), while **NTLX** increased the least (`rel_change=29.86`).

And Finally, **MSFT** experimented the lowest fall in price (`max_fall=-0.37`), while **NFLX** saw its price plummet `76%` at some point during this period.

Overall, it seems that this wasn't a good period to be invested in **NFLX**!

### 3.4. Visualizing yearly price movements through candlestick charts

A [candlestick chart](https://en.wikipedia.org/wiki/Candlestick_chart) is a style of financial chart used to describe price movements of a security, derivative, or currency. It is similar to a bar chart in that each candlestick represents all four important pieces of information for that day: open and close in the thick body; high and low in the “candle wick”. Being densely packed with information, it tends to represent trading patterns over short periods of time, often a few days or a few trading sessions.


In [20]:
fig = candlestick_yearly(tickers_data, ticker_selected)
fig.update_layout(template="seaborn").show()

We can see that for **GOOG**, there was a positive trend in price from 2019 to 2021, and then the price plummeted in 2022, with some slight recovery in 2023 so far.

### 3.5. Visualizing price seasonality

[Seasonality](https://www.investopedia.com/terms/s/seasonality.asp) is a characteristic of a time series in which the data experiences regular and predictable changes that recur every calendar year. Any predictable fluctuation or pattern that recurs or repeats over a one-year period is said to be seasonal.


In [21]:
fig = violin_monthly(tickers_data, ticker_selected)
fig.update_layout(template="seaborn").show()

It seems that **GOOG** price tends to be lower during the summer months compared to the rest of the year.

In [22]:
fig = violin_weekday(tickers_data, ticker_selected)
fig.update_layout(template="seaborn").show()

There is no noticeable difference in the price of **GOOG** depending on the day of the week.

## 4. Data Modelling

In this section, we will model our time-series data for price forecasting. We start by selecting one ticker to perform forecasting on.


### 4.1. Perform last pre-processing step (as detailed before in section 3)

Leading and trailing `NaN` values are removed, whereas `NaN` values found in the middle of periods, such as weekends, will be forward filled. In the case of stock securities of publicly traded companies, we don't have values for Saturdays and Sundays, since the stock markets are closed.

In [23]:
# 0. Setup
ticker_data = deepcopy(
    tickers_data.reorder_levels(order=[1, 0], axis=1)[ticker_selected]
)

# 1. Ensure index is datetime with the right frequency
ticker_data.index = pd.to_datetime(ticker_data.index, format="YYYY-MM-DD")
ticker_data = ticker_data.asfreq("D").sort_index()

# 2. Discard missing values at the beginning and end of the time period
ticker_data = ticker_data.loc[
    ticker_data.first_valid_index() : ticker_data.last_valid_index()
]

# 3. Forward fill missing values
ticker_data = ticker_data.fillna(method="ffill")

# 4. Add date-based exogenous variables
for date_part in ("day", "weekday", "month", "year"):
    ticker_data[date_part] = getattr(ticker_data.index, date_part)

### 4.2. Split data into training and testing sets

Data is split into training and testing sets for model fitting and evaluation, respectively. Given the variability of the stock market, we will only be using the last year of available data.

In [24]:
start_train_date = tickers_data.index.max() - relativedelta(years=1)
test_steps = 28
train_data, test_data = (
    ticker_data.loc[
        start_train_date : (ticker_data.index.max() - relativedelta(days=test_steps))
    ],
    ticker_data.iloc[-test_steps:, :],
)

In [25]:
fig = go.Figure(
    [
        go.Scatter(
            name="Training",
            x=train_data.index,
            y=train_data["Adj Close"],
            mode="lines",
        ),
        go.Scatter(
            name="Testing",
            x=test_data.index,
            y=test_data["Adj Close"],
            mode="lines",
        ),
    ]
).update_layout(
    yaxis_title="Adj. Close price",
    xaxis_title="Date",
    legend_title="Data",
)
fig.update_layout(template="seaborn").show()

### 4.3. Model selection and hyper-parameter tuning

A LightGBM autoregressor model is trained to make predictions into the future. Its hyper-parameters are tuned using a special case of Grid Search with backtesting. We use the `Adj Close` column as the best estimate for the daily closing price.

Mean Squared Error (MSE), a common regression evaluation metric, is used in this case to assess performance.

In [26]:
pred_col = "Adj Close"
exog_cols = ["day", "month"]

In [27]:
# 1. Define forecaster model
forecaster = ForecasterAutoreg(
    regressor=LGBMRegressor(random_state=random_seed),
    lags=14,
)

# 2. Define hyper-parameter grids to configure
lags_grid = [1, 2, 3, 7, 14, 28]
param_grid = {"n_estimators": [50, 100, 200], "max_depth": [None, 3, 7, 11]}

# 3. Tune hyper-parameters using grid search
results_grid = grid_search_forecaster(
    forecaster=forecaster,
    y=train_data[pred_col],
    exog=train_data[exog_cols],
    param_grid=param_grid,
    lags_grid=lags_grid,
    steps=test_steps,
    refit=True,
    metric="mean_squared_error",
    initial_train_size=int(len(train_data) * 0.5),
    return_best=True,
    verbose=False,
)

# 4. Make predictions
pred = forecaster.predict(steps=test_steps, exog=test_data[exog_cols])

# 5. Get test error
error_mse = mean_squared_error(y_true=test_data[pred_col], y_pred=pred)

Number of models compared: 72.


loop lags_grid: 100%|███████████████████████████████████████| 6/6 [00:21<00:00,  3.50s/it]

`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [1 2 3 4 5 6 7] 
  Parameters: {'max_depth': 7, 'n_estimators': 50}
  Backtesting metric: 24.09972050081255



#### 4.3.1. Table comparing the performance of the grid search

In [28]:
results_grid.head()

,lags,params,mean_squared_error,max_depth,n_estimators
42,"[1, 2, 3, 4, 5, 6, 7]","{'max_depth': 7, 'n_estimators': 50}",24.099721,7.0,50.0
45,"[1, 2, 3, 4, 5, 6, 7]","{'max_depth': 11, 'n_estimators': 50}",24.099721,11.0,50.0
36,"[1, 2, 3, 4, 5, 6, 7]","{'max_depth': None, 'n_estimators': 50}",24.099721,NaN,50.0
39,"[1, 2, 3, 4, 5, 6, 7]","{'max_depth': 3, 'n_estimators': 50}",24.700616,3.0,50.0
30,"[1, 2, 3]","{'max_depth': 7, 'n_estimators': 50}",25.770382,7.0,50.0


The lowest mean squared error was achieved with `lags=7` and the following hyper-parameters:

`{'max_depth': 7, 'n_estimators': 50}`

### 4.4. Model Evaluation

In [29]:
fig = go.Figure(
    [
        go.Scatter(
            name="Testing",
            x=test_data.index,
            y=test_data["Adj Close"],
            mode="lines",
        ),
        go.Scatter(
            name="Predicted",
            x=pred.index,
            y=pred,
            mode="lines",
        ),
    ]
).update_layout(
    yaxis_title="Adj. Close price",
    xaxis_title="Date",
    legend_title="Data",
)
fig.update_layout(template="seaborn").show()

In [30]:
print(f"MSE: {error_mse}")

MSE: 101.26721783889442


### 4.5. Conclusion and future improvements

The predicted price was very far from the actual trend, suggesting that more effort is needed in order to achieve a better performance. Improvements include using more powerful models, such as **XGBoost**, more exhaustive hyper-parameter tuning, as well as a more careful selection of training and testing data. The latter might prove crucial due to the great variability experimented in time.

In any case, and as mentioned at the beginning of this notebook, security price is hardly dependent on past performance. Instead, it is driven by current events from varying sources, from sentiments expressed in social media to geopolotical changes. Including this information as exogenous variables would greatly increase our ability to predict future prices.

### 4.6. Acknowledgments

This project was done as part of the [Data Science Nanodegree Program at Udacity](https://www.udacity.com/course/data-scientist-nanodegree--nd025).

The documentation of the [skforecast](https://pypi.org/project/skforecast/0.3.0/) Python package was fundamental in learning how to analyse and forecat financial data.